# TP2 - Análise de hiper-parâmetros no PSO
Aluno: Pedro Augusto de Castro e Castro

Para este trabalho foram desenvolvidos dois algoritmos para otimização, o Global Best PSO (gbest PSO) e o Local Best PSO em anel (lbest PSO). Para avaliar o desempenho de cada um, foram escolhidas duas funções de benchmarking, uma unimodal (sphere) e a outra multimodal (rastrigin). Além disso, este trabalho visa comparar os fatores de inércia (w) e de constrição (X). 



In [1]:
import random
import math
import sys
!{sys.executable} -m pip install tabulate;
from tabulate import tabulate

nv = 10 # número de variáveis
qtd = 31 # numero de execuções
initial_fitness = float("inf") 
bounds = [(-100, 100) for _ in range(nv)]

iterations = 25 #10
particle_size = int(100000 / iterations)    

def objective_function_sphere(xx):
    y = sum([xi ** 2 for xi in xx])
    return y

def objective_function_rastrigin(xx):
    y = 10 * len(xx) + sum([xi ** 2 - 10 * math.cos(2 * math.pi * xi) for xi in xx])
    return y

mean = lambda x: sum(x) / len(x)

std_dev = lambda y: math.sqrt(
    sum(
        list(
            map(lambda x: (x - mean(y)) ** 2, y)
        )
    ) / len(y))
     

Uma classe chamada Particle foi implementada, de forma a simular uma única partícula no PSO. Essa classe guarda a posição atual da partícula, a sua melhor posição local, sua velocidade, além de possuir métodos para avaliar a função objetivo e atualizar a velocidade e posição.  

In [2]:
class Particle:
    def __init__(self,bounds):
        self.particle_position=[]                     # posição da partícula
        self.particle_velocity=[]                     # velocidade da partícula
        self.local_best_particle_position=[]          # melhor posição da partícula
        self.fitness_local_best_particle_position= initial_fitness  # valor da função objetivo inicial da partícula com a melhor posição
        self.fitness_particle_position=initial_fitness             # valor da função objetivo inicial da partícula
 
        for i in range(nv):
            self.particle_position.append(random.uniform(bounds[i][0],bounds[i][1])) # gerar uma posição inicial aleatória
            self.particle_velocity.append(random.uniform(-1,1)) # gerar uma velocidade aleatória
 
    def evaluate(self,objective_function):
        self.fitness_particle_position=objective_function(self.particle_position)           
        if self.fitness_particle_position < self.fitness_local_best_particle_position:
            self.local_best_particle_position=self.particle_position                  # update o melhor local
            self.fitness_local_best_particle_position=self.fitness_particle_position  # update o fitness do melhor local

    def update_velocity(self, global_best_particle_position):
        for i in range(nv):
            r1=random.random()
            r2=random.random() 
            cognitive_velocity = c1*r1*(self.local_best_particle_position[i] - self.particle_position[i])
            social_velocity = c2*r2*(global_best_particle_position[i] - self.particle_position[i])
            self.particle_velocity[i] = X*(w*self.particle_velocity[i] + cognitive_velocity + social_velocity)
 
    def update_position(self,bounds):
        for i in range(nv):
            self.particle_position[i]=self.particle_position[i]+self.particle_velocity[i]
 
            # checar e arrumar para satisfazer os limites superiores
            if self.particle_position[i]>bounds[i][1]:
                self.particle_position[i]=bounds[i][1]
            # checar e arrumar para satisfazer os limites inferiores
            if self.particle_position[i] < bounds[i][0]:
                self.particle_position[i]=bounds[i][0]
                

A implementação de ambos os códigos PSO (lbest e gbest) são apresentadas abaixo. Para o lbest, foi definida uma vizinhança em anel, onde cada partícula se comunicava somente com seus 2 vizinhos imediatos. Sendo assim, o Global Best variava de partícula para partícula. Já no gbest PSO a vizinhança de uma partícula era toda a população, e sendo assim, o Global Best era igual para todas.

In [3]:
class lbest_PSO():
    def __init__(self,objective_function,bounds,particle_size,iterations):
 
        self.fitness_global_best_particle_position=initial_fitness
        self.global_ring_best_particle_position=[[] * nv] * particle_size
        self.global_best_particle_position=[]
 
        swarm_particle=[]
        for i in range(particle_size):
            swarm_particle.append(Particle(bounds))
            
        self.GB=[]         
        for i in range(iterations):
            for j in range(particle_size):
                swarm_particle[j].evaluate(objective_function)
                
            for j in range(particle_size):
                first_index = (j - 1) % len(swarm_particle)
                last_index = (j + 1) % len(swarm_particle)

                neigh_ring = [swarm_particle[first_index], 
                              swarm_particle[j], 
                              swarm_particle[last_index]]                    

                ind = list(map(lambda x: x.fitness_local_best_particle_position, neigh_ring)).index(
                    min(list(map(lambda x: x.fitness_local_best_particle_position, neigh_ring))))

                self.global_ring_best_particle_position[j] = neigh_ring[ind].particle_position

                if neigh_ring[ind].fitness_local_best_particle_position < self.fitness_global_best_particle_position:                        
                    self.global_best_particle_position = neigh_ring[ind].particle_position
                    self.fitness_global_best_particle_position = \
                        float(neigh_ring[ind].fitness_local_best_particle_position)
            for j in range(particle_size):
                swarm_particle[j].update_velocity(self.global_ring_best_particle_position[j])
                swarm_particle[j].update_position(bounds)
                 

    def gbest(self):
        return self.fitness_global_best_particle_position


In [4]:
class gbest_PSO():
    def __init__(self,objective_function,bounds,particle_size,iterations):
 
        self.fitness_global_best_particle_position=initial_fitness
        self.global_best_particle_position=[]
 
        swarm_particle=[]
        for i in range(particle_size):
            swarm_particle.append(Particle(bounds))
            
        self.GB=[]         
        for i in range(iterations):
            for j in range(particle_size):
                swarm_particle[j].evaluate(objective_function)
                if swarm_particle[j].fitness_particle_position < self.fitness_global_best_particle_position:
                    self.global_best_particle_position = list(swarm_particle[j].particle_position)
                    self.fitness_global_best_particle_position = float(swarm_particle[j].fitness_particle_position)
            for j in range(particle_size):
                swarm_particle[j].update_velocity(self.global_best_particle_position)
                swarm_particle[j].update_position(bounds)
                 
            self.GB.append(self.fitness_global_best_particle_position) # grava o melhor fitness


    def gbest(self):
        return self.fitness_global_best_particle_position
    

Definição da função para calcular a média e o desvio padrão dos casos a serem estudados:

In [5]:
def calc_mean_dev(objective_function):
    lbest = []
    gbest = []
    print(f"Configuração: X = {X}, w = {w}")
    for i in range(qtd):
        var_lbest = lbest_PSO(objective_function,bounds,particle_size,iterations)
        lbest.append(var_lbest.gbest())    
        var_gbest = gbest_PSO(objective_function,bounds,particle_size,iterations)
        gbest.append(var_gbest.gbest()) 
    mean_lbest = mean(lbest)
    std_dev_lbest = std_dev(lbest)
    mean_gbest = mean(gbest)
    std_dev_gbest = std_dev(gbest)
    return mean_lbest, std_dev_lbest, mean_gbest, std_dev_gbest

# Case 1: X = 1.0 e w = 1.0

In [6]:
X = 1.0
w = 1.0

## Sphere function:

In [7]:
 # Main PSO  - Sphere
c1 = 1.655 #0.3
c2 = 1.655 #0.2

mean_lbest_sphere_case1, std_dev_lbest_sphere_case1, mean_gbest_sphere_case1, std_dev_gbest_sphere_case1 = calc_mean_dev(objective_function_sphere)

print(f'lbest PSO Sphere => valor médio = {mean_lbest_sphere_case1:.2f}, desvio padrão = {std_dev_lbest_sphere_case1:.2f}')
print(f'gbest PSO Sphere => valor médio = {mean_gbest_sphere_case1:.2f}, desvio padrão = {std_dev_gbest_sphere_case1:.2f}')



Configuração: X = 1.0, w = 1.0
lbest PSO Sphere => valor médio = 3688.39, desvio padrão = 949.44
gbest PSO Sphere => valor médio = 1352.92, desvio padrão = 373.41


## Rastrigin function:

In [8]:
# Main PSO  - Rastrigin
c1 = 1.655
c2 = 1.655

mean_lbest_rastrigin_case1, std_dev_lbest_rastrigin_case1, mean_gbest_rastrigin_case1, std_dev_gbest_rastrigin_case1 = calc_mean_dev(objective_function_rastrigin)

print(f'lbest PSO Rastrigin => valor médio = {mean_lbest_rastrigin_case1:.2f}, desvio padrão = {std_dev_lbest_rastrigin_case1:.2f}')
print(f'gbest PSO Rastrigin => valor médio = {mean_gbest_rastrigin_case1:.2f}, desvio padrão = {std_dev_gbest_rastrigin_case1:.2f}')



Configuração: X = 1.0, w = 1.0
lbest PSO Rastrigin => valor médio = 3389.93, desvio padrão = 950.48
gbest PSO Rastrigin => valor médio = 1498.93, desvio padrão = 401.18


# Case 2: X = 1.0 e w = 0.7298

In [9]:
X = 1.0
w = 0.7298

## Sphere function:

In [10]:
 # Main PSO  - Sphere
c1 = 1.655
c2 = 1.655

mean_lbest_sphere_case2, std_dev_lbest_sphere_case2, mean_gbest_sphere_case2, std_dev_gbest_sphere_case2 = calc_mean_dev(objective_function_sphere)

print(f'lbest PSO Sphere => valor médio = {mean_lbest_sphere_case2:.2f}, desvio padrão = {std_dev_lbest_sphere_case2:.2f}')
print(f'gbest PSO Sphere => valor médio = {mean_gbest_sphere_case2:.2f}, desvio padrão = {std_dev_gbest_sphere_case2:.2f}')


Configuração: X = 1.0, w = 0.7298
lbest PSO Sphere => valor médio = 3070.04, desvio padrão = 666.91
gbest PSO Sphere => valor médio = 59.23, desvio padrão = 26.03


## Rastrigin function:

In [11]:
# Main PSO  - Rastrigin
c1 = 1.655
c2 = 1.655

mean_lbest_rastrigin_case2, std_dev_lbest_rastrigin_case2, mean_gbest_rastrigin_case2, std_dev_gbest_rastrigin_case2 = calc_mean_dev(objective_function_rastrigin)

print(f'lbest PSO Rastrigin => valor médio = {mean_lbest_rastrigin_case2:.2f}, desvio padrão = {std_dev_lbest_rastrigin_case2:.2f}')
print(f'gbest PSO Rastrigin => valor médio = {mean_gbest_rastrigin_case2:.2f}, desvio padrão = {std_dev_gbest_rastrigin_case2:.2f}')



Configuração: X = 1.0, w = 0.7298
lbest PSO Rastrigin => valor médio = 3233.07, desvio padrão = 704.30
gbest PSO Rastrigin => valor médio = 160.84, desvio padrão = 31.16


# Caso 3: X = 0.7298 e w = 1

In [12]:
X = 0.7298
w = 1.0

## Sphere function:

In [13]:
 # Main PSO  - Sphere
c1 = 1.655
c2 = 1.655

mean_lbest_sphere_case3, std_dev_lbest_sphere_case3, mean_gbest_sphere_case3, std_dev_gbest_sphere_case3 = calc_mean_dev(objective_function_sphere)

print(f'lbest PSO Sphere => valor médio = {mean_lbest_sphere_case3:.2f}, desvio padrão = {std_dev_lbest_sphere_case3:.2f}')
print(f'gbest PSO Sphere => valor médio = {mean_gbest_sphere_case3:.2f}, desvio padrão = {std_dev_gbest_sphere_case3:.2f}')


Configuração: X = 0.7298, w = 1.0
lbest PSO Sphere => valor médio = 2480.56, desvio padrão = 405.83
gbest PSO Sphere => valor médio = 20.58, desvio padrão = 8.03


## Rastrigin function:

In [14]:
# Main PSO  - Rastrigin
c1 = 1.655
c2 = 1.655

mean_lbest_rastrigin_case3, std_dev_lbest_rastrigin_case3, mean_gbest_rastrigin_case3, std_dev_gbest_rastrigin_case3 = calc_mean_dev(objective_function_rastrigin)

print(f'lbest PSO Rastrigin => valor médio = {mean_lbest_rastrigin_case3:.2f}, desvio padrão = {std_dev_lbest_rastrigin_case3:.2f}')
print(f'gbest PSO Rastrigin => valor médio = {mean_gbest_rastrigin_case3:.2f}, desvio padrão = {std_dev_gbest_rastrigin_case3:.2f}')


Configuração: X = 0.7298, w = 1.0
lbest PSO Rastrigin => valor médio = 2522.14, desvio padrão = 745.15
gbest PSO Rastrigin => valor médio = 98.96, desvio padrão = 16.09


# Caso 4: X = 0.7298 e w = 0.7298

In [15]:
X = 0.7298
w = 0.7298

## Sphere function:

In [16]:
 # Main PSO  - Sphere
c1 = 1.655
c2 = 1.655

mean_lbest_sphere_case4, std_dev_lbest_sphere_case4, mean_gbest_sphere_case4, std_dev_gbest_sphere_case4 = calc_mean_dev(objective_function_sphere)

print(f'lbest PSO Sphere => valor médio = {mean_lbest_sphere_case4:.2f}, desvio padrão = {std_dev_lbest_sphere_case4:.2f}')
print(f'gbest PSO Sphere => valor médio = {mean_gbest_sphere_case4:.2f}, desvio padrão = {std_dev_gbest_sphere_case4:.2f}')


Configuração: X = 0.7298, w = 0.7298
lbest PSO Sphere => valor médio = 902.17, desvio padrão = 284.97
gbest PSO Sphere => valor médio = 0.07, desvio padrão = 0.05


## Rastrigin function:

In [17]:
# Main PSO  - Rastrigin
c1 = 1.655 #0.7
c2 = 1.655#0.1

mean_lbest_rastrigin_case4, std_dev_lbest_rastrigin_case4, mean_gbest_rastrigin_case4, std_dev_gbest_rastrigin_case4 = calc_mean_dev(objective_function_rastrigin)

print(f'lbest PSO Rastrigin => valor médio = {mean_lbest_rastrigin_case4:.2f}, desvio padrão = {std_dev_lbest_rastrigin_case4:.2f}')
print(f'gbest PSO Rastrigin => valor médio = {mean_gbest_rastrigin_case4:.2f}, desvio padrão = {std_dev_gbest_rastrigin_case4:.2f}')


Configuração: X = 0.7298, w = 0.7298
lbest PSO Rastrigin => valor médio = 1057.87, desvio padrão = 288.23
gbest PSO Rastrigin => valor médio = 31.03, desvio padrão = 8.36


# Resultados

## Sphere:

In [18]:
print(tabulate([['X = 1, w = 1', f"{mean_lbest_sphere_case1:.2f} " + u"\u00B1" f" {std_dev_lbest_sphere_case1:.2f}" , 
                 f"{mean_gbest_sphere_case1:.2f} " + u"\u00B1" f" {std_dev_gbest_sphere_case1:.2f}"], 
               
                ['X = 1, 0 < w < 1', f"{mean_lbest_sphere_case2:.2f} " + u"\u00B1" f" {std_dev_lbest_sphere_case2:.2f}" , 
                 f"{mean_gbest_sphere_case2:.2f} " + u"\u00B1" f" {std_dev_gbest_sphere_case2:.2f}"],
               
                ['0 < X < 1, w = 1', f"{mean_lbest_sphere_case3:.2f} " + u"\u00B1" f" {std_dev_lbest_sphere_case3:.2f}" , 
                 f"{mean_gbest_sphere_case3:.2f} " + u"\u00B1" f" {std_dev_gbest_sphere_case3:.2f}"],
               
                ['0 < X < 1, 0 < w < 1', f"{mean_lbest_sphere_case4:.2f} " + u"\u00B1" f" {std_dev_lbest_sphere_case4:.2f}" , 
                 f"{mean_gbest_sphere_case4:.2f} " + u"\u00B1" f" {std_dev_gbest_sphere_case4:.2f}"]], 
               
               headers=['Configuração', 'Local Best (em anel)', 'Global Best']))

Configuração          Local Best (em anel)    Global Best
--------------------  ----------------------  ----------------
X = 1, w = 1          3688.39 ± 949.44        1352.92 ± 373.41
X = 1, 0 < w < 1      3070.04 ± 666.91        59.23 ± 26.03
0 < X < 1, w = 1      2480.56 ± 405.83        20.58 ± 8.03
0 < X < 1, 0 < w < 1  902.17 ± 284.97         0.07 ± 0.05


## Rastrigin:

In [19]:
print(tabulate([['X = 1, w = 1', f"{mean_lbest_rastrigin_case1:.2f} " + u"\u00B1" f" {std_dev_lbest_rastrigin_case1:.2f}" , 
                 f"{mean_gbest_rastrigin_case1:.2f} " + u"\u00B1" f" {std_dev_gbest_rastrigin_case1:.2f}"], 
               
                ['X = 1, 0 < w < 1', f"{mean_lbest_rastrigin_case2:.2f} " + u"\u00B1" f" {std_dev_lbest_rastrigin_case2:.2f}" , 
                 f"{mean_gbest_rastrigin_case2:.2f} " + u"\u00B1" f" {std_dev_gbest_rastrigin_case2:.2f}"],
               
                ['0 < X < 1, w = 1', f"{mean_lbest_rastrigin_case3:.2f} " + u"\u00B1" f" {std_dev_lbest_rastrigin_case3:.2f}" , 
                 f"{mean_gbest_rastrigin_case3:.2f} " + u"\u00B1" f" {std_dev_gbest_rastrigin_case3:.2f}"],
               
                ['0 < X < 1, 0 < w < 1', f"{mean_lbest_rastrigin_case4:.2f} " + u"\u00B1" f" {std_dev_lbest_rastrigin_case4:.2f}" , 
                 f"{mean_gbest_rastrigin_case4:.2f} " + u"\u00B1" f" {std_dev_gbest_rastrigin_case4:.2f}"]], 
               
               headers=['Configuração', 'Local Best (em anel)', 'Global Best']))

Configuração          Local Best (em anel)    Global Best
--------------------  ----------------------  ----------------
X = 1, w = 1          3389.93 ± 950.48        1498.93 ± 401.18
X = 1, 0 < w < 1      3233.07 ± 704.30        160.84 ± 31.16
0 < X < 1, w = 1      2522.14 ± 745.15        98.96 ± 16.09
0 < X < 1, 0 < w < 1  1057.87 ± 288.23        31.03 ± 8.36


Observando os resultados apresentados anteriormente, pode-se perceber que o gbest PSO obteve resultados superiores àqueles obtidos com o lbest. Além disso, em ambas implementações, o caso onde X e w foram menor que 1 apresentou resultados mais próximos do mínimo global real. Isso ocorre pois esses fatores reduzem as velocidades das partículas a fim de se atingir a convergência.

Como apresentado em [5], o peso de inércia do PSO (w) foi introduzido por [6], e ele tem como objetivo controlar as habilidades de "exploration" e "exploitation" do enxame. Ele controla o momentum da partícula, diminuindo a contribuição da velocidade anterior. Com w menor que 1, a velocidade das partículas tendem a diminuir até a convergência.

O fator de contrição (X) também tem como objetivo controlar as habilidades de "exploration" e "exploitation" do enxame, melhorando, assim, o tempo de convergência e a qualidade da solução encontrada. Baixos valores de w e X indicam "exploitation", enquanto que valores mais altos resultam em "exploration" [5]. O modelo com o fator de contrição garante a convergência sob dadas restrições.

Foi possível observar, a partir dos resultados, que utilizando um fator de contrição (X) menor que um, e mantendo a inércia (w) igual a um, foi melhor que o contrário (X igual a um e w menor que um). Ao se fazer  X ou w menores que um, já é possível perceber uma melhor convergência do algoritmo, em relação ao caso 1, sem nenhuma restrição.

Além disso, como era de se esperar, os resultados para a função de Esfera foram melhores do que para a função Rastrigin. Isso era esperado pois trata-se de uma função unimodal contra uma multimodal.





# Bibliografia
[1] J. J. Liang, B-Y. Qu, P. N. Suganthan, Alfredo G. Hern´andez-D´ıaz, ”Problem Definitions and Evaluation Criteria for the CEC 2013 Special Session and Competition on Real-Parameter Optimization”, Technical Report 201212, Computational Intelligence Laboratory, Zhengzhou University, Zhengzhou China and Technical Report, Nanyang Technological University, Singapore, January 2013.

[2] LI, Xiaodong. Niching without niching parameters: particle swarm optimization using a ring topology. IEEE Transactions on Evolutionary Computation, v. 14, n. 1, p. 150-169, 2009.

[3] SENGUPTA, Saptarshi; BASAK, Sanchita; PETERS, Richard Alan. Particle Swarm Optimization: A survey of historical and recent developments with hybridization perspectives. Machine Learning and Knowledge Extraction, v. 1, n. 1, p. 157-191, 2019.

[4] SERANI, A. et al. On the use of synchronous and asynchronous single-objective deterministic particle swarm optimization in ship design problems. In: Proceeding of OPT-i, International Conference on Engineering and Applied Sciences Optimization, Kos Island, Greece. 2014.

[5] Frederico Gadelha Guimarães, "Particle Swarm Optimization", EEE882 - Evolutionary Computation. Machine Intelligence and Data Science (MINDS) Lab

[6] EBERHART, R. C. ; SHI, Y. ; KENNEDY, J. Swarm Intelligence. Burlington,
MA, USA : Morgan Kaufmann, 2001.